In [1]:
# Box Office Mojo - daily data on top wide releases yearly in 2021 and 2022

# import data libraries
import pandas as pd
import numpy as np
from datetime import date, datetime
import pytz

# import selenium/webscraping libs
from urllib.request import urlopen
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

# Access webdriver for Chrome
driver_path = '/Users/zacharywong/Downloads/chromedriver'
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# keep track of day/time
est = pytz.timezone('US/Eastern')
fmt = '%d/%m/%Y %H:%M:%S'
now = datetime.now()
now = now.astimezone(est).strftime(fmt)
print("day/time: " + now)

urlYearly = 'https://www.boxofficemojo.com/year/?ref_=bo_nb_hm_secondarytab'
columns = ["Date", "DOW", "Rank", "Daily", "%+/-YR", "%+/-LW", "Theaters", "Avg", "To Date", "Day"]

day/time: 08/07/2022 00:07:06


In [2]:
# Filter by In Year Releases and Wide Releases
def clickFilters():
    XPATHinYear= "/html/body/div[1]/main/div/div/div[1]/div[2]/span/form/span/select"
    XPATHwideRelease = "/html/body/div[1]/main/div/div/div[1]/div[3]/span/form/span/select"
    
    try:
        filterInYearSelect = Select(driver.find_element(By.XPATH, XPATHinYear))
    except:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, XPATHinYear)))
        filterInYearSelect = Select(driver.find_element(By.XPATH, XPATHinYear))
        
    filterInYearSelect.select_by_visible_text("In-year releases")
    
    try:
        filterwidereleaseSelect = Select(driver.find_element(By.XPATH, XPATHwideRelease))
    except:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, XPATHwideRelease)))
        filterwidereleaseSelect = Select(driver.find_element(By.XPATH, XPATHwideRelease))
        
    filterwidereleaseSelect.select_by_visible_text("Wide releases")
        

In [3]:
def getMovieLink(index):
    XPATHMovie = "/html/body/div[1]/main/div/div/div[3]/div/table[2]/tbody/tr[2]/td[{index}]/a".format(index = index)
    try: 
        movie = driver.find_element(By.XPATH, XPATHMovie)
    except: 
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, XPATHMovie)))
        movie = driver.find_element(By.XPATH, XPATHMovie)
    return movie 

In [4]:
def getRowColumnNumber():
    adjustForHeader = 1
    rowXPATH = "//*[@id='table']/div/table[2]/tbody/tr"
    colXPATH = "//*[@id='table']/div/table[2]/tbody/tr[1]/th"
    rowNumber = len(driver.find_elements(By.XPATH, rowXPATH))-adjustForHeader
    colNumber = len(driver.find_elements(By.XPATH, colXPATH))
    return rowNumber, colNumber


In [5]:
def getMovieData():
    rowNumber, colNumber = getRowColumnNumber()
    indexList = []
    
    for number in range(0, rowNumber):
        indexList.append(number)
    
    tempResult = pd.DataFrame(index = indexList , columns = columns)
    adjustRow = 2
    adjustCol = 1
    for row in range(0, rowNumber):
        for col in range(0, colNumber-1):
            #print(rowNumber, columns[colNumber])
            #print(row, col)
            #print(rowNumber, colNumber)
            if col == 0 or col == 1:
                XPATHValue = "//*[@id='table']/div/table[2]/tbody/tr[{row}]/td[{col}]/a".format(row = row+adjustRow, col = col + adjustCol)
                value = driver.find_element(By.XPATH, XPATHValue)
            else:
                XPATHValue = "//*[@id='table']/div/table[2]/tbody/tr[{row}]/td[{col}]".format(row = row+adjustRow, col = col + adjustCol)
                value = driver.find_element(By.XPATH, XPATHValue)
            #print(rowNumber, columns[colNumber])
            tempResult.at[row, columns[col]] = value.text
    return tempResult
        
                

In [6]:
def run():
    result = pd.DataFrame(columns = columns)
    print(result)
    driver.get(urlYearly)
    #XPATH = "/html/body/div[1]/main/div/div/div[3]/div/table[2]/tbody/tr[2]/td[1]/a"
    XPATH_2022 = "//*[@class='a-link-normal' and text() = '2022']"
    try:
        yearListDriver = driver.find_element(By.XPATH, XPATH_2022)
    except:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, XPATH_2022)))
        yearListDriver = driver.find_element(By.XPATH, XPATH_2022)
    yearListDriver.click()
    clickFilters()
    movieDriver = getMovieLink(2)
    movieDriver.click()
    getRowColumnNumber()
    #print(tableDriver)
    tempResult = getMovieData()
    tempResult
    #result = result.append(tempResult)
    #result

In [7]:
run()
#result.head()

Empty DataFrame
Columns: [Date, DOW, Rank, Daily, %+/-YR, %+/-LW, Theaters, Avg, To Date, Day]
Index: []
      Date        DOW Rank        Daily  %+/-YR  %+/-LW Theaters      Avg  \
0   May 27     Friday    1  $52,012,661       -       -    4,735  $10,984   
1   May 28   Saturday    1  $38,017,701  -26.9%       -    4,735   $8,029   
2   May 29     Sunday    1  $36,677,097   -3.5%       -    4,735   $7,745   
3   May 30     Monday    1  $33,807,521   -7.8%       -    4,735   $7,139   
4   May 31    Tuesday    1  $15,800,746  -53.3%       -    4,735   $3,337   
5    Jun 1  Wednesday    1  $14,821,417   -6.2%       -    4,735   $3,130   
6    Jun 2   Thursday    1  $14,466,868   -2.4%       -    4,735   $3,055   
7    Jun 3     Friday    1  $25,006,817  +72.9%  -51.9%    4,751   $5,263   
8    Jun 4   Saturday    1  $36,008,145    +44%   -5.3%    4,751   $7,579   
9    Jun 5     Sunday    1  $29,022,049  -19.4%  -20.9%    4,751   $6,108   
10   Jun 6     Monday    1  $12,402,196  -57.3% 